In [3]:
#We can go into our root file and see what Trees are availiable
%matplotlib inline
import sys, os
if __package__ is None:
    import sys, os
    sys.path.append(os.path.realpath("/data/shared/Software/"))
    sys.path.append(os.path.realpath("../../"))
p = "/home/dweitekamp/mpi_learn/"
if(not p in sys.path):
    sys.path.append(p)
import numpy as np
import pandas as pd
import ntpath
import glob
import deepconfig

#from keras.utils.visualize_util import plot
#from IPython.display import Image, display

from CMS_Deep_Learning.preprocessing.preprocessing import *
from CMS_Deep_Learning.callbacks import OverfitStopping, SmartCheckpoint
from CMS_Deep_Learning.storage.batch import batchAssertArchived, batchExecuteAndTestTrials
from CMS_Deep_Learning.storage.archiving import *
from CMS_Deep_Learning.storage.MPIArchiving import *
from CMS_Deep_Learning.postprocessing.analysistools import findsubsets
from CMS_Deep_Learning.layers.lorentz import Lorentz, _lorentz
from CMS_Deep_Learning.layers.slice import Slice

from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Flatten, Reshape, Activation, Dropout, Convolution2D, merge, Input, Flatten, Lambda, LSTM, Masking
from keras.engine.topology import Layer
from keras.callbacks import EarlyStopping
from keras.utils.visualize_util import plot
from keras.layers.advanced_activations import LeakyReLU


dc = deepconfig.deepconfig(gpu='gpu0', backend='theano')




using gpu0
using theano


In [7]:
#The observables taken from the table
observ_types = ['E/c', 'Px', 'Py', 'Pz', 'PT_ET','Eta', 'Phi', 'Charge', 'X', 'Y', 'Z',\
                     'Dxy', 'Ehad', 'Eem', 'MuIso', 'EleIso', 'ChHadIso','NeuHadIso','GammaIso', "ObjType"]
vecsize = len(observ_types)
epochs = 30
batch_size = 100

label_dir_pairs = \
            [   ("ttbar", "/data/shared/Delphes/ttbar_lepFilter_13TeV/pandas_h5/"),
                ("wjet", "/data/shared/Delphes/wjets_lepFilter_13TeV/pandas_h5/"),
                ("qcd", "/data/shared/Delphes/qcd_lepFilter_13TeV/pandas_h5/")
            ]


In [8]:
#Find all the subsets of label_dir_pairs and store them as sorted lists
#ldpsubsets = []
ldpsubsets = [sorted(list(s)) for s in findsubsets(label_dir_pairs)]
#Make sure that we do 3-way classification as well
ldpsubsets.append(label_dir_pairs)
#ldpsubsets=label_dir_pairs


In [ ]:
def genModel(name,object_profiles,out_dim, depth, lstm_activation="relu", lstm_dropout = 0.0, dropout=0.0,output_activation="softmax"):
    inputs = []
    mergelist = []
    for i, profile in enumerate(object_profiles):
        inp = a = Input(shape=(profile.max_size + 1*(profile.punctuation != None), vecsize), name="input_"+str(i))
        inputs.append(inp)
        mergelist.append(a)
    a = merge(mergelist,mode='concat',concat_axis=1, name="merge")
    for i in range(depth):
        a = Masking(mask_value=0.0)(a)
        a = LSTM(vecsize,
                 input_shape=(None,vecsize),
                 dropout_W=lstm_dropout,
                 dropout_U=lstm_dropout,
                 activation=lstm_activation,
                 name = "lstm_" +str(i))(a)
        if(dropout > 0.0):
            a =  Dropout(dropout, name="dropout_"+str(i))(a)
    dense_out = Dense(out_dim, activation=output_activation, name='main_output')(a)
    model = Model(input=inputs, output=dense_out, name=name)
    return model

In [ ]:
archive_dir = "/data/shared/Delphes/keras_archive/"
patience = 8
earlyStopping = EarlyStopping(verbose=1, patience=patience)
trial_tups = []
output_activation="softmax"
#Loop over all subsets
print(ldpsubsets)
for ldp in ldpsubsets:
    labels = [x[0] for x in ldp]
    for sort_on in ["PT_ET", "Phi", "Eta"]:
    #for sort_on in ["Phi"]:
        #Use object maxes from Find_Maxes_From Query
        object_profiles = [ObjectProfile("Electron",-1, pre_sort_columns=["PT_ET"], pre_sort_ascending=False, sort_columns=[sort_on], sort_ascending=False, addColumns={"ObjType":1}),
                            ObjectProfile("MuonTight", -1, pre_sort_columns=["PT_ET"], pre_sort_ascending=False, sort_columns=[sort_on], sort_ascending=False, addColumns={"ObjType":2}),
                            #ObjectProfile("Photon", -1, pre_sort_columns=["PT_ET"], pre_sort_ascending=False, sort_columns=[sort_on], sort_ascending=False, addColumns={"ObjType":3}),
                            ObjectProfile("MissingET", 1, addColumns={"ObjType":4}),
                            ObjectProfile("EFlowPhoton",100, pre_sort_columns=["PT_ET"], pre_sort_ascending=False, sort_columns=[sort_on], sort_ascending=False, addColumns={"ObjType":5}), 
                            ObjectProfile("EFlowNeutralHadron",100, pre_sort_columns=["PT_ET"], pre_sort_ascending=False, sort_columns=[sort_on], sort_ascending=False, addColumns={"ObjType":6}), 
                            ObjectProfile("EFlowTrack",100, pre_sort_columns=["PT_ET"], pre_sort_ascending=False, sort_columns=[sort_on], sort_ascending=False, addColumns={"ObjType":7})]  

        resolveProfileMaxes(object_profiles, ldp)

        dps, l = getGensDefaultFormat(archive_dir, (20000,75000), 95000, \
                             object_profiles,ldp,observ_types,batch_size=batch_size,megabytes=100, verbose=0)

        dependencies = batchAssertArchived(dps)
        val,   num_val   = l[0]
        train, num_train = l[1]
        #test,  num_test  = l[2]
        max_q_size = l[2]
        print("MAXQ: ",max_q_size)

       
        for name in ['LSTM']:
            for depth in [1]:
                    for activation in ['tanh']:
                        for lstm_dropout in [0.0]:
                            for dropout in [0.0]:
                                activation_name = activation if isinstance(activation, str) \
                                                    else activation.__name__

                                model = genModel(name,object_profiles, len(labels), depth, activation, lstm_dropout, dropout,output_activation=output_activation)
                                
                                train_dps = train.args[0]
                                val_dps = val.args[0]

                                trial = MPI_KerasTrial(archive_dir, name=name, model=model)
                                #trial = KerasTrial(archive_dir, name=name, model=model)

                                trial.setTrain(train_procedure=train_dps,
                                               samples_per_epoch=num_train
                                              )
                                trial.setValidation(val_procedure=val_dps,
                                                   nb_val_samples=num_val)
                                
                                trial.setCompilation(loss='categorical_crossentropy',
                                          optimizer='rmsprop',
                                          metrics=['accuracy']
                                              )

                                trial.setFit_Generator( 
                                                nb_epoch=epochs,
                                                callbacks=[earlyStopping],
                                                max_q_size = max_q_size)
                                trial.write()


#                                print("EXECUTE: ", name,labels, depth, activation_name)
#                                trial.execute(custom_objects={"Lorentz":Lorentz,"Slice": Slice},
#                                             train_arg_decode_func=label_dir_pairs_args_decoder,
#                                             val_arg_decode_func=label_dir_pairs_args_decoder)


#                                trial.test(test_proc=test,
#                                             test_samples=num_test,
#                                             custom_objects={"Lorentz":Lorentz,"Slice": Slice},
#                                            arg_decode_func = label_dir_pairs_args_decoder)

                                trial_tups.append((trial, None, None, dependencies))

                                trial.to_record({"labels": labels,
                                                 "depth": depth,
                                                 "sort_on" : sort_on,
                                                 "activation": activation_name,
                                                 "dropout":dropout,
                                                 "lstm_dropout":lstm_dropout,
                                                 "query": None,
                                                 "patience" : patience,
                                                 "useObjTypeColumn" : True,
                                                 "output_activation" : output_activation
                                                 #"Non_MPI" :True
                                                })
#trial_tups[0][0].summary()
#trial_tups[0][0].execute()
for tup in trial_tups:
    tup[0].summary()
for tup in trial_tups:
    tup[0].execute()
#batchExecuteAndTestTrials(trial_tups)
                
        

    

[[('ttbar', '/data/shared/Delphes/ttbar_lepFilter_13TeV/pandas_h5/'), ('wjet', '/data/shared/Delphes/wjets_lepFilter_13TeV/pandas_h5/')], [('qcd', '/data/shared/Delphes/qcd_lepFilter_13TeV/pandas_h5/'), ('ttbar', '/data/shared/Delphes/ttbar_lepFilter_13TeV/pandas_h5/')], [('qcd', '/data/shared/Delphes/qcd_lepFilter_13TeV/pandas_h5/'), ('wjet', '/data/shared/Delphes/wjets_lepFilter_13TeV/pandas_h5/')], [('ttbar', '/data/shared/Delphes/ttbar_lepFilter_13TeV/pandas_h5/'), ('wjet', '/data/shared/Delphes/wjets_lepFilter_13TeV/pandas_h5/'), ('qcd', '/data/shared/Delphes/qcd_lepFilter_13TeV/pandas_h5/')]]
Starting batchAssertArchived...
Done.('MAXQ: ', 34.0)
Starting batchAssertArchived...
Done.('MAXQ: ', 34.0)
Starting batchAssertArchived...
Done.('MAXQ: ', 34.0)
Starting batchAssertArchived...
Done.('MAXQ: ', 34.0)
Starting batchAssertArchived...
Done.('MAXQ: ', 34.0)
Starting batchAssertArchived...
Done.('MAXQ: ', 34.0)
Starting batchAssertArchived...
Done.('MAXQ: ', 34.0)
Starting batchAs

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5005)


Failed to load weights at /data/shared/Delphes/keras_archive/blobs/f6d05/72b3f72fbd228eb0b13a44f6499cf5c4101/weights.h5
Starting batchAssertArchived...


Using Theano backend.
Using gpu device 1: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5005)


Done.Starting batchAssertArchived...
Done.Failed to load history at /data/shared/Delphes/keras_archive/blobs/f6d05/72b3f72fbd228eb0b13a44f6499cf5c4101/history.json
Files for worker 1:
  /data/shared/Delphes/keras_archive/blobs/e2b44/e12042761a82ecd016cce4ea4da609e8bb5/archive.h5
  /data/shared/Delphes/keras_archive/blobs/b2912/a32f9291b84b41d4d552cd0d212e431a107/archive.h5
  /data/shared/Delphes/keras_archive/blobs/7e123/3b05a6f6c9227566eea2e2cffff6d58fdb5/archive.h5
  /data/shared/Delphes/keras_archive/blobs/698d8/c5c7cc8ea7ba022e19ad0632bb0d8ae8824/archive.h5
  /data/shared/Delphes/keras_archive/blobs/0e8db/4e735ec440700657c66539fe34fb65ec1fb/archive.h5
  /data/shared/Delphes/keras_archive/blobs/3bc24/a2117e4fd3b0cae35c8e3d8535167987d1e/archive.h5
  /data/shared/Delphes/keras_archive/blobs/70945/1f9cbcecef4fe39b4e09246a4b90d628649/archive.h5
  /data/shared/Delphes/keras_archive/blobs/b92cb/12c5dcde7f2291c8d4e47a6cc0237b804a0/archive.h5
  /data/shared/Delphes/keras_archive/blobs/31731

In [ ]:
#trial = KerasTrial.find_by_hashcode(archive_dir,"f6d0572b3f72fbd228eb0b13a44f6499cf5c4101")
#print(t)
#trial.remove_from_archive()

# 